In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNNV2, RNN, RNNV12
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager, select_label
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
model_loc = "./train_log/separate/read3-noglps-lmd-0/all-stages-max-micro-auc.tfmodel"
log_dir = './train_log/separate/D1-read3-mtd/'
logger.set_logger_dir(log_dir, action='d')

[1105 08:34:14 @logger.py:74] Argv: /home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1007/jupyter/kernel-5d4da6c1-e5fd-4d18-86c4-95e6d1278021.json


In [2]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "max-macro_f1.tfmodel"

In [3]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

In [5]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.stages = [2]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
dm.get_num_info()

{'test': (762, 3237), 'train': (692, 3016), 'val': (156, 572)}

In [6]:
dm.get_imbalance_ratio()

,train,val,test
anlage in statu nascendi,7.439024,7.666667,7.758621
cellular blastoderm,1.005797,1.025974,1.005263
dorsal ectoderm anlage in statu nascendi,6.440860,6.090909,6.398058
faint ubiquitous,5.653846,5.782609,4.953125
maternal,3.907801,4.379310,4.013158
posterior endoderm anlage in statu nascendi,15.878049,14.600000,15.565217
procephalic ectoderm anlage in statu nascendi,7.650000,7.666667,7.758621
subset,3.552632,3.457143,3.590361
ubiquitous,0.870270,0.714286,0.809976
ventral ectoderm anlage in statu nascendi,7.650000,6.800000,7.282609


In [7]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.4
config.gamma = 2
config.use_glimpse = False
config.use_hidden_dense = False
config.read_time = 3
config.batch_size = 64

threshold = 0.4
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
#val_data = dm.get_test_stream()

In [9]:
np.set_printoptions(formatter={'float_kind': lambda x: '%.2f' % x})
model = RNN(config, is_finetuning=False, label_weights=dm.get_imbalance_ratio().train.values)
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4),  
                                                                           (40, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, 
                               ignore=ignore_restore),
                           max_epoch=1, tower=[0])
Trainer(train_config).train()

[1105 08:35:16 @inference_runner.py:82] InferenceRunner will eval on an InputSource of size 2
[1105 08:35:16 @input_source.py:180] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1105 08:35:16 @training.py:90] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
[1105 08:35:21 @model_utils.py:47] Model Parameters: 
name                            shape                  dim
-----------------------

[1105 08:35:26 @base.py:212] Creating the session ...
[1105 08:35:28 @base.py:216] Initializing the session ...
[1105 08:35:28 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1105 08:35:30 @base.py:223] Graph Finalized.
[1105 08:35:30 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1105 08:35:30 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1105 08:35:32 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1105 08:35:32 @base.py:257] Start Epoch 1 ...


100%|##########|10/10[00:32<00:00, 0.51it/s]

[1105 08:36:04 @base.py:267] Epoch 1 (global_step 10) finished, time:32.19 sec.



  0%|          |0/2[00:00<?,?it/s]/home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|2/2[00:14<00:00, 0.14it/s]


[1105 08:36:20 @saver.py:90] Model saved to ./train_log/separate/D1-read3-mtd/model-10.
[1105 08:36:21 @saver.py:159] Model with maximum 'micro_auc' saved.
[1105 08:36:21 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 13
[1105 08:36:21 @monitor.py:362] QueueInput/queue_size: 2.8828
[1105 08:36:21 @monitor.py:362] coverage: 3.8594
[1105 08:36:21 @monitor.py:362] learning_rate: 0.0001
[1105 08:36:21 @monitor.py:362] loss/value_1: 0.56055
[1105 08:36:21 @monitor.py:362] macro_auc: 0.75422
[1105 08:36:21 @monitor.py:362] macro_f1: 0.14321
[1105 08:36:21 @monitor.py:362] mean_average_precision: 0.43125
[1105 08:36:21 @monitor.py:362] micro_auc: 0.74742
[1105 08:36:21 @monitor.py:362] micro_f1: 0.52674
[1105 08:36:21 @monitor.py:362] one_error: 0.40625
[1105 08:36:21 @monitor.py:362] ranking_loss: 0.24713
[1105 08:36:21 @monitor.py:362] ranking_mean_average_precision: 0.70596
[1105 08:36:21 @monitor.py:362] training_ap: 0.37496
[1105 08:36:21 @monitor.py:362] training_au

In [9]:
tf.get_collection(tf.GraphKeys.MODEL_VARIABLES)

[<tf.Variable 'resnet_v2_101/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/weights:0' shape=(1, 1, 64, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/co

In [17]:
model = RNN(config, is_finetuning=False)
tf.reset_default_graph()
pred_config = PredictConfig(model=model,
                            session_init=SaverRestore(
                                model_path=log_dir + save_name),
                            output_names=['logits_export', 'label'],
                            )
# pred = Predictor(pred_config, test_data, nr_proc=2, ordered=False)
pred = SimpleDatasetPredictor(pred_config, test_data)

accumulator = seq(pred.get_result()) \
    .smap(lambda a, b: (a.shape[0], calcu_metrics(a, b, config.validation_metrics, threshold))) \
    .aggregate(Accumulator(*config.validation_metrics), lambda accu, args: accu.feed(args[0], *args[1]))
metrics = accumulator.retrive()
print(metrics)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
[1104 15:51:24 @collection.py:133] New collections created in : tf.GraphKeys.MODEL_VARIABLES, resnet_v2_101_end_points
[1104 15:51:24 @collection.py:152] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2), (tf.GraphKeys.UPDATE_OPS: 0->2)


ResourceExhaustedError: OOM when allocating tensor with shape[3,3,1024,512]
	 [[Node: custom_cnn/conv1/weights/Assign = Assign[T=DT_FLOAT, _class=["loc:@custom_cnn/conv1/weights"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](custom_cnn/conv1/weights, custom_cnn/conv1/weights/Initializer/truncated_normal)]]

Caused by op 'custom_cnn/conv1/weights/Assign', defined at:
  File "/home/yangyang/Applications/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yangyang/Applications/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/yangyang/Application

In [ ]:
import pickle

In [ ]:
with open